<a href="https://colab.research.google.com/github/indianspeedster/pytorch/blob/master/tf_to_tensorrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python3 -m pip install --upgrade tensorrt
!python3 -m pip install --upgrade tensorrt_lean
!python3 -m pip install --upgrade tensorrt_dispatch

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1-py2.py3-none-any.whl size=16972 sha256=0d060d72d4a66b1686e5e5ec16c21f7caca2b92a55eef451edc46d1f8674c84f
  Stored in directory: /root/.cache/pip/wheels/6d/29/56/abdffd4c604f255b5254bef3f1c598ab7811ea020540599438
Successfully built tensorrt


In [3]:
import tensorrt
assert tensorrt.Builder(tensorrt.Logger())

8.6.1


### Without tensor Rt implementation

In [36]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import sys
import time
import numpy as np

# Navigate to correct position in filesystem
#script_directory = os.path.dirname(os.path.abspath(sys.argv[0]))
#os.chdir(script_directory)

# Set up the model
INPUT_IMG_SIZE = 224
INPUT_IMG_SHAPE = (224, 224, 3)
model = tf.keras.applications.MobileNetV2(
  input_shape=INPUT_IMG_SHAPE
)
@tf.function
def serve(x):
  return model(x, training=False)


# Prepare and pass the input image
image_path = 'parrot.jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((INPUT_IMG_SIZE, INPUT_IMG_SIZE), Image.BICUBIC)
input_data = np.array(img)/255.0
input_data = input_data.reshape(1, INPUT_IMG_SIZE, INPUT_IMG_SIZE, 3)

# First prediction is slow, we won't count it
output = serve(input_data)

# Now, start counting!
start_time = time.time()

# Make a prediction!
output = serve(input_data)

# Get and print the result
inf_time =  time.time() - start_time
print(f"time: {inf_time}s" )

top_3 = np.argsort(output.numpy().squeeze())[-3:][::-1]
url = tf.keras.utils.get_file(
    'ImageNetLabels.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(url).read().splitlines())[1:]

for i in top_3:
    print('{:.6f}'.format(output.numpy()[0, i]), ':',  imagenet_labels[i])

time: 0.007984638214111328s
0.875274 : macaw
0.005244 : bee eater
0.004900 : flamingo


#### Save the above model

In [1]:
SAVED_MODEL_DIR="./models/native_saved_model"
model.save(SAVED_MODEL_DIR)

#### Import and build tensorrt

In [ ]:
import tensorrt
assert tensorrt.Builder(tensorrt.Logger())

#### convert the model for tensorrt inference

In [4]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

converter = trt.TrtGraphConverterV2(
   input_saved_model_dir=SAVED_MODEL_DIR,
   precision_mode=trt.TrtPrecisionMode.FP16
)

trt_func = converter.convert()
converter.summary()

TRTEngineOP Name                 Device        # Nodes # Inputs      # Outputs     Input DTypes       Output Dtypes      Input Shapes       Output Shapes     
TRTEngineOp_000_000              device:GPU:0  421     1             1             ['float32']        ['float32']        [[-1, 224, 224 ... [[-1, 1000]]      

	- AddV2: 10x
	- BiasAdd: 1x
	- Const: 264x
	- Conv2D: 35x
	- DepthwiseConv2dNative: 17x
	- FusedBatchNormV3: 52x
	- MatMul: 1x
	- Mean: 1x
	- Pad: 4x
	- Relu6: 35x
	- Softmax: 1x

[*] Total number of TensorRT engines: 1
[*] % of OPs Converted: 99.53% [421/423]



#### build the model to specify the input shape

In [9]:
import tensorflow as tf
import numpy as np
data_float32 = np.array(input_data, dtype=np.float32)

def input_fn():
    yield [data_float32]

converter.build(input_fn=input_fn)

#### Save the tftrt model

In [34]:
OUTPUT_SAVED_MODEL_DIR="./models/tftrt_saved_model"
converter.save(output_saved_model_dir=OUTPUT_SAVED_MODEL_DIR)

#### Make inference

In [41]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import sys
import time
import numpy as np
from tensorflow.python.compiler.tensorrt import trt_convert as trt

# Navigate to correct position in filesystem
#script_directory = os.path.dirname(os.path.abspath(sys.argv[0]))
#os.chdir(script_directory)

# Set up the model
INPUT_IMG_SIZE = 224
INPUT_IMG_SHAPE = (224, 224, 3)
loaded_model = tf.saved_model.load(OUTPUT_SAVED_MODEL_DIR)
infer = loaded_model.signatures['serving_default']


# Prepare and pass the input image
image_path = 'parrot.jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((INPUT_IMG_SIZE, INPUT_IMG_SIZE), Image.BICUBIC)
input_data = np.array(img)/255.0
input_data = input_data.reshape(1, INPUT_IMG_SIZE, INPUT_IMG_SIZE, 3)
data_float32 = np.array(input_data, dtype=np.float32)
image_tensor = tf.convert_to_tensor(data_float32)
# First prediction is slow, we won't count it
output = infer(image_tensor)

# Now, start counting!
start_time = time.time()

# Make a prediction!
output = infer(image_tensor)

# Get and print the result
inf_time =  time.time() - start_time
print(f"time: {inf_time}s" )

top_3 = np.argsort(output["predictions"].numpy().squeeze())[-3:][::-1]
url = tf.keras.utils.get_file(
    'ImageNetLabels.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(url).read().splitlines())[1:]

for i in top_3:
    print('{:.6f}'.format(output["predictions"].numpy()[0, i]), ':',  imagenet_labels[i])

time: 0.003282785415649414s
0.877158 : macaw
0.005335 : bee eater
0.004814 : flamingo
